In [1]:
import pandas as pd

# I. Data loading and preparation

In [10]:
# loading thesaurus
thesaurus = pd.read_csv('1. Data/thesaurus_key_words.csv', encoding="ISO-8859-1", sep=';')
thesaurus.head()

,classification_E,catégorie,symptome-fr,symptome-en,CIM_10,CIM11,Orphanet
0,E1,Période néonatale,Encéphalopathie myoclonique précoce,Benign familial neonatal epilepsy (BFNE),G40.8,8A61.0Y,1935.0
1,E2,Période néonatale,Epilepsie néonatale familiale bénigne (BFNE),Early myoclonic encephalopathy (EME),G40.8,8A61.10,1949.0
2,E3,Période néonatale,Syndrome d'ohtahara,Ohtahara syndrome,G40.8,8A62.Y,1934.0
3,E31,Nourrisons,Encépahlopathie myoclonique des affections non...,Myoclonic encephalopathy in nonprogressive dis...,G40.4,8A62.Y,86913.0
4,E33,Nourrisons,Epilepsie benigne du nourisson,Benign infantile epilepsy,G40.3,8A61.1Z,166302.0


In [11]:
# Loading link database
database =  pd.read_csv('1. Data/Database_links.csv')
database.head()

,Exam_name,Patient_name,RR_intervals_filepath,Feats_filepath,Annotations_filepath,Patient_infos_filepath
0,00004671_s006_t000,4671,./../TU_EEG_Seizures_processed/v0_4/res-v0_4/d...,./../TU_EEG_Seizures_processed/v0_4/feats-v0_4...,./../TU_EEG_Seizures_processed/v0_4/annot-v0_4...,./../TU_EEG_Seizures_processed/v0_4/info-patie...
1,00004671_s008_t000,4671,./../TU_EEG_Seizures_processed/v0_4/res-v0_4/d...,./../TU_EEG_Seizures_processed/v0_4/feats-v0_4...,./../TU_EEG_Seizures_processed/v0_4/annot-v0_4...,./../TU_EEG_Seizures_processed/v0_4/info-patie...
2,00004671_s011_t006,4671,./../TU_EEG_Seizures_processed/v0_4/res-v0_4/d...,./../TU_EEG_Seizures_processed/v0_4/feats-v0_4...,./../TU_EEG_Seizures_processed/v0_4/annot-v0_4...,./../TU_EEG_Seizures_processed/v0_4/info-patie...
3,00004671_s011_t005,4671,./../TU_EEG_Seizures_processed/v0_4/res-v0_4/d...,./../TU_EEG_Seizures_processed/v0_4/feats-v0_4...,./../TU_EEG_Seizures_processed/v0_4/annot-v0_4...,./../TU_EEG_Seizures_processed/v0_4/info-patie...
4,00004671_s011_t003,4671,./../TU_EEG_Seizures_processed/v0_4/res-v0_4/d...,./../TU_EEG_Seizures_processed/v0_4/feats-v0_4...,./../TU_EEG_Seizures_processed/v0_4/annot-v0_4...,./../TU_EEG_Seizures_processed/v0_4/info-patie...


In [12]:
# Keeping only relevant columns of the database
database = database[['Exam_name', 'Patient_name','Patient_infos_filepath']]

In [13]:
# Let's see an example of the the file path
print(database['Patient_infos_filepath'][0])

./../TU_EEG_Seizures_processed/v0_4/info-patients-v0_4/edf/dev/01_tcp_ar/046/00004671/s006_2010_10_06/00004671_s006.txt


In [14]:
# We have to adapt the link for proper use by changing './../TU_EEG_Seizures_processed/v0_4/info-patients-v0_4' for 'CR_Patients_info_patients-v0_4'

database['Patient_infos_filepath'] = database['Patient_infos_filepath'].apply(lambda x: '1. Data/CR_Patients_info_patients-v0_4' + x[len('./../TU_EEG_Seizures_processed/v0_4/info-patients-v0_4'):])
database.head()

,Exam_name,Patient_name,Patient_infos_filepath
0,00004671_s006_t000,4671,1. Data/CR_Patients_info_patients-v0_4/edf/dev...
1,00004671_s008_t000,4671,1. Data/CR_Patients_info_patients-v0_4/edf/dev...
2,00004671_s011_t006,4671,1. Data/CR_Patients_info_patients-v0_4/edf/dev...
3,00004671_s011_t005,4671,1. Data/CR_Patients_info_patients-v0_4/edf/dev...
4,00004671_s011_t003,4671,1. Data/CR_Patients_info_patients-v0_4/edf/dev...


In [15]:
# Are some reports there several times?
print('Unique values:')
print(len(database['Patient_infos_filepath'].unique()))
print('Number total of values:')
print(database.shape[0])


Unique values:
1423
Number total of values:
5610


Lots of reports are there several times. We will therefore create a new dataframe with only unique values

In [16]:
classification_dataset = pd.DataFrame(database['Patient_infos_filepath'].unique(), columns=['filepath'])
print(classification_dataset.shape)
classification_dataset.head()

(1423, 1)


,filepath
0,1. Data/CR_Patients_info_patients-v0_4/edf/dev...
1,1. Data/CR_Patients_info_patients-v0_4/edf/dev...
2,1. Data/CR_Patients_info_patients-v0_4/edf/dev...
3,1. Data/CR_Patients_info_patients-v0_4/edf/dev...
4,1. Data/CR_Patients_info_patients-v0_4/edf/dev...


In [17]:
# Let's explore the first 5 files in particular
for i in range(5):
    print('_________' +str(i) +'_________')
    sample_file = pd.read_csv(classification_dataset['filepath'][i], sep='\n', header=None)
    print(sample_file)
    print('__________________')

_________0_________
                                           0
0                               Description:
1  2.5 to 5 hz spike/wave and polyspike wave
2                                 Gen’l slow
3                                   Abnormal
__________________
_________1_________
                                                   0
0  LENGTH OF THE RECORDING:  22 minutes and 53 se...
1  CLINICAL HISTORY:  The patient is a 27-year-ol...
2  MEDICATIONS:  Ativan, Dilantin, clonazepam, ch...
3  INTRODUCTION:  Digital video EEG is performed ...
4  DESCRIPTION OF THE RECORD:  During maximal wak...
5                                          FINDINGS:
6                           1. Mild diffuse slowing.
7  2. Runs of non-evolving frontocentral theta ac...
8  3. Isolated diffuse frontally predominant slow...
9  IMPRESSION:  This is a very abnormal EEG due t...
__________________
_________2_________
                                                   0
0  MEDICATIONS:  Vimpat, Norvasc, Felbama

Files are more or less complete but it seems we could make index with the structure (History, Introduction, etc). But for the moment, we will only load all of them concatenated to focus on the words only.

In [18]:
# Let's create of function to merge all text into one

def load_and_merge(filepath):
    try:
        report = pd.read_csv(filepath, sep= '\n', header=None, encoding='ISO-8859-1')
        merged_report = ''
        for i in report.index:
            merged_report += ' ' + report[0][i]
        return merged_report
    except:
        print('Error for: ', filepath)
        return 'Not loaded'

# We will load it in the database
%time

classification_dataset['report'] = classification_dataset['filepath'].apply(lambda x: load_and_merge(x))

print('\n')
print('____________________')
print('\n')
print(classification_dataset.head())

CPU times: user 0 ns, sys: 0 ns, total: 0 ns
Wall time: 6.91 µs
Error for:  1. Data/CR_Patients_info_patients-v0_4/edf/train/02_tcp_le/051/00005103/s002_2009_06_04/00005103_s002.txt


____________________


                                            filepath  \
0  1. Data/CR_Patients_info_patients-v0_4/edf/dev...   
1  1. Data/CR_Patients_info_patients-v0_4/edf/dev...   
2  1. Data/CR_Patients_info_patients-v0_4/edf/dev...   
3  1. Data/CR_Patients_info_patients-v0_4/edf/dev...   
4  1. Data/CR_Patients_info_patients-v0_4/edf/dev...   

                                              report  
0   Description: 2.5 to 5 hz spike/wave and polys...  
1   LENGTH OF THE RECORDING:  22 minutes and 53 s...  
2   MEDICATIONS:  Vimpat, Norvasc, Felbamate, Car...  
3   CLINICAL HISTORY:  27 year old gentleman with...  
4   MEDICATIONS:  Vimpat, Norvasc, Felbamate, Car...  


There was only an error for 1 file, we will go on and fix it for production later.

In [21]:
# Now, we will save the dataset for later use
classification_dataset.to_csv('2. Results/Classification_dataset.csv', index=False)